In [4]:
import pandas as pd

In [5]:
data=pd.read_excel(r"C:\scada-docker\Swat Data\Normal  Data 2023\22June2020 (1).xlsx")
data.head()

,t_stamp,P1_STATE,LIT101.Pv,FIT101.Pv,MV101.Status,P101.Status,P102.Status,P2_STATE,FIT201.Pv,AIT201.Pv,...,FIT601.Pv,P601.Status,P602.Status,P603.Status,LSH601.Alarm,LSL601.Alarm,LSH602.Alarm,LSL602.Alarm,LSH603.Alarm,LSL603.Alarm
0,2020-06-22 10:00:00.000,3,695.284100,0.0,1,1,1,2,0.000256,18.072288,...,0.000256,1,1,1,Inactive,Inactive,Active,Inactive,Inactive,Active
1,2020-06-22 10:00:01.000,3,695.127100,0.0,1,1,1,2,0.000256,18.104332,...,0.000256,1,1,1,Inactive,Inactive,Active,Inactive,Inactive,Active
2,2020-06-22 10:00:02.005,3,694.930847,0.0,1,1,1,2,0.000256,18.104332,...,0.000256,1,1,1,Inactive,Inactive,Active,Inactive,Inactive,Active
3,2020-06-22 10:00:03.010,3,694.930847,0.0,1,1,1,2,0.000256,18.104332,...,0.000256,1,1,1,Inactive,Inactive,Active,Inactive,Inactive,Active
4,2020-06-22 10:00:04.015,3,694.852300,0.0,1,1,1,2,0.000256,18.104332,...,0.000256,1,1,1,Inactive,Inactive,Active,Inactive,Inactive,Active


In [6]:
from kafka import KafkaConsumer
import json


In [ ]:

consumer = KafkaConsumer(
    'scada.actuators_v2',
    bootstrap_servers='localhost:9092',
    auto_offset_reset='earliest',
    group_id='peek-consumer',
    value_deserializer=lambda v: json.loads(v.decode('utf-8'))
)

for i, msg in enumerate(consumer):
    print(json.dumps(msg.value, indent=2))
    if i >= 2:  # just show 10 messages
        break

consumer.close()


In [2]:
from influxdb_client import InfluxDBClient

client = InfluxDBClient(
    url="http://localhost:8086",
    token="nt-scada-token-secret-key-12345",
    org="nt-scada"
)
query_api = client.query_api()

query = '''
import "influxdata/influxdb/schema"
schema.measurements(bucket: "scada_data_v2")
'''

tables = query_api.query(org="nt-scada", query=query)
for table in tables:
    for record in table.records:
        print(record.values["_value"])



actuator
actuator_data
kafka_data


In [17]:
# Query the normal_data measurement
query_normal_data = '''
from(bucket: "scada_data_v2")
  |> range(start: 1970-01-01T00:00:00Z)  // include all data
  |> filter(fn: (r) => r._measurement == "normal_data")
  |> limit(n: 10)  // show first 10 rows; remove limit for all rows
'''

tables = query_api.query(org="nt-scada", query=query_normal_data)

print("=== normal_data rows ===")
for table in tables:
    for record in table.records:
        print(record.values)



=== normal_data rows ===


In [18]:
import pandas as pd

# Query normal_data measurement
query_normal_data = '''
from(bucket: "scada_data_v2")
  |> range(start: 1970-01-01T00:00:00Z)
  |> filter(fn: (r) => r._measurement == "normal_data")
'''

tables = query_api.query(org="nt-scada", query=query_normal_data)

# Collect all rows
rows = []
for table in tables:
    for record in table.records:
        row = record.values.copy()
        # Remove keys starting with _ except _time (optional)
        row = {k: v for k, v in row.items() if not k.startswith('_') or k == "_time"}
        rows.append(row)

# Load into pandas DataFrame
df = pd.DataFrame(rows)

# Display first 10 rows
print(df.head(10))


Empty DataFrame
Columns: []
Index: []
